In [1]:
import sys

sys.path.append('..')

import decision_transformer.manage_time as DT_manager
%matplotlib ipympl

cuda


In [2]:
# Simulation configuration
transformer_model_name = 'checkpoint_ff_time_chunk100_ctgrtg'
import_config = DT_manager.transformer_import_config(transformer_model_name)

In [3]:
# Import dataset and dataloader
datasets, dataloaders = DT_manager.get_train_val_test_data(mdp_constr=import_config['mdp_constr'], dataset_scenario=import_config['dataset_scenario'], timestep_norm=import_config['timestep_norm'], chunksize=import_config['chunksize'])
train_dataset, val_dataset, test_dataset = datasets
train_loader, eval_loader, test_loader = dataloaders


Loading data from root/dataset/torch/...Almost done, DATA IS NOT SHUFFLED YET.

Completed, DATA IS SHUFFLED.



In [4]:
# Get the model and set it into eval mode
model = DT_manager.get_DT_model(transformer_model_name, train_loader, eval_loader)
model.eval();

GPT size: 11.5M parameters


In [ ]:
# Sample from test dataloader
import numpy as np
import torch
data_stats = test_loader.dataset.data_stats
'''state_init = np.array([0.2,0.15+0.2,0,0,0,0])#np.array([0.22, 2.14, 1.53, 0, 0, 0])#np.array([1.58643470e-01,  5.06685234e-01, -1.07279630e+00, 0.0,0.0,0.0])#np.array([0.29490604, 2.34616295, 1.69898474,0,0,0])#
state_final = np.array([3.2,2.3,np.pi,0,0,0])#np.array([3.31, 0.60, -1.28, 0, 0, 0])#np.array([3.25586406,  2.29016048, -0.80894559, 0.0, 0.0, 0.0])#np.array([3.15600952, 0.25998125, 0.99323413, 0,0,0])#
test_sample = next(iter(test_loader))
data_stats = test_loader.dataset.data_stats
test_sample[0][0,:,:] = (torch.tensor(np.repeat(state_init[None,:], 200, axis=0)) - data_stats['states_mean'])/(data_stats['states_std'] + 1e-6)#(torch.tensor(xs[:-1,:]) - data_stats['states_mean'])/(data_stats['states_std'] + 1e-6)#
test_sample[1][0,:,:] = torch.zeros((200,3))#(torch.tensor(us) - data_stats['actions_mean'])/(data_stats['actions_std'] + 1e-6)#
test_sample[2][0,:,0] = torch.zeros((200,))#torch.from_numpy(compute_reward_to_go(test_sample[1][0,:,:]))#
test_sample[3][0,:,0] = torch.zeros((200,))#torch.from_numpy(compute_constraint_to_go(test_sample[0][0,:,:].cpu().numpy(), obs_positions, obs_radii))#
test_sample[4][0,:,:] = (torch.tensor(np.repeat(state_final[None,:], 200, axis=0)) - data_stats['goal_mean'])/(data_stats['goal_std'] + 1e-6)'''
test_sample = test_loader.dataset.getix(27471)#next(iter(test_loader))# <- deterministic sample OR random sample -> next(iter(test_loader))#28040, 27471, 31454
states_i, actions_i, rtgs_i, ctgs_i, ttgs_i, goal_i, timesteps_i, attention_mask_i, time_discr_i, time_sec_i, ix_i = test_sample
x_init = (states_i[0,0,:] * data_stats['states_std'][0]) + (data_stats['states_mean'][0])
x_final = (goal_i[0,0,:] * data_stats['goal_std'][0]) + (data_stats['goal_mean'][0])
ttgs0 = (ttgs_i[0,:] * data_stats['ttgs_std'][0]) + (data_stats['ttgs_mean'][0])

(x_init, x_final, ttgs0)

In [ ]:
# Use the model just to perform imitation learning
print(test_sample[-1])
DT_trajectory = DT_manager.use_model_for_imitation_learning(model, test_loader, test_sample, rtg_perc=1.0, ctg_perc=0., ttg=44, use_dynamics=True, ctg_clipped=True, chunksize=import_config['chunksize'])

In [ ]:
dyn_trajectory, _ = DT_manager.torch_model_inference_dyn(model, test_loader, test_sample, rtg_perc=1.0, ctg_perc=0., ttg=44, ctg_clipped=True, chunksize=import_config['chunksize'], end_on_ttg=True)

In [ ]:
DT_trajectory['xypsi_ol'] = dyn_trajectory['xypsi_dyn']
DT_trajectory['dv_ol'] = dyn_trajectory['dv_dyn']
DT_trajectory['ttgs_ol'] = dyn_trajectory['ttgs_dyn']

In [ ]:
# Print the index of the trajectory and plot
print(test_sample[-1])
DT_manager.plot_DT_trajectory(DT_trajectory)

In [ ]:
# Use the model to perform offline RL
# Take the desired rtg0 from convex problem
from dynamics.freeflyer import FreeflyerModel, ocp_no_obstacle_avoidance, ocp_obstacle_avoidance, np
ffm = FreeflyerModel(verbose=True)
traj, J, n_iter, status = ocp_no_obstacle_avoidance(ffm,x_init.cpu().numpy(),x_final.cpu().numpy())
xs = traj['states']
us = traj['actions_G']
rtg_0 = -np.sum((np.linalg.norm(us, ord=1, axis=0)))

# Use the model (open-loop / dyn-in-the-loop)
ol_trajectory, runtime_DT = DT_manager.torch_model_inference_ol(model, test_loader, test_sample, rtg_perc=1., ctg_perc=0., rtg=rtg_0, ctg_clipped=True)
dyn_trajectory, runtime_DT = DT_manager.torch_model_inference_dyn(model, test_loader, test_sample, rtg_perc=1., ctg_perc=0., rtg=rtg_0, ctg_clipped=True)

In [ ]:
# Print the index of the trajectory and plot
DT_trajectory = {
    'xypsi_true' : ((states_i[0,:,:].cpu() * data_stats['states_std']) + data_stats['states_mean']).T,
    'xypsi_dyn' : dyn_trajectory['xypsi_dyn'],
    'xypsi_ol' : ol_trajectory['xypsi_ol'],
    'dv_true' : ((actions_i[0,:,:].cpu() * data_stats['actions_std']) + data_stats['actions_mean']).T,
    'dv_dyn' : dyn_trajectory['dv_dyn'],
    'dv_ol' : ol_trajectory['dv_ol'],
    'time' : ol_trajectory['time']
}
print(test_sample[-1])
DT_manager.plot_DT_trajectory(DT_trajectory)

In [ ]:
import decision_transformer.ros_manage as ros_TTO_manager

ros_model = ros_TTO_manager.get_only_DT_model(transformer_model_name, 6, 3)
data_stats = np.load('saved_files/data_stats.npz',allow_pickle=True)['data_stats'].item()
rosdyn_trajectory, rosruntime_DT = ros_TTO_manager.ros_model_inference_dyn(model, data_stats, x_init.cpu().numpy(), x_final.cpu().numpy(), rtg_perc=1., ctg_perc=0., rtg=rtg_0, ctg_clipped=True)

In [ ]:
DT_trajectory = {
    'xypsi_true' : ((states_i[0,:,:].cpu() * data_stats['states_std']) + data_stats['states_mean']).T,
    'xypsi_dyn' : rosdyn_trajectory['xypsi_dyn'],
    'xypsi_ol' : dyn_trajectory['xypsi_dyn'],
    'dv_true' : ((actions_i[0,:,:].cpu() * data_stats['actions_std']) + data_stats['actions_mean']).T,
    'dv_dyn' : rosdyn_trajectory['dv_dyn'],
    'dv_ol' : dyn_trajectory['dv_dyn'],
    'time' : dyn_trajectory['time']
}
print(test_sample[-1])
DT_manager.plot_DT_trajectory(DT_trajectory)

From this point onward -> experimental stuff

In [5]:
from dynamics.freeflyer_time import *
x_init = np.array([0.19273452, 0.18552349, 2.39470136, 0, 0, 0])
x_final = np.array([3.22172603, 1.76696898, 1.80025613, 0, 0, 0])
qm = FreeflyerModel(verbose=True)
traj_opt, J, scp_iter, status = ocp_no_obstacle_avoidance(qm,x_init,x_final,20.0)
traj_scp, J, status, scp_iter = ocp_obstacle_avoidance(qm, traj_opt['states'], traj_opt['actions_G'], x_init, x_final)

Initializing freeflyer class.
[solve]: Problem infeasible. [obstacle avoidance]: False


/home/davide.celestini/Transformer/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:
import torch
# Sample from test dataloader
data_stats = test_loader.dataset.data_stats
test_sample = test_loader.dataset.getix(0)# <- deterministic sample OR random sample -> next(iter(test_loader))#
'''states_i, actions_i, rtgs_i, ctgs_i, goal_i, timesteps_i, attention_mask_i, time_discr_i, time_sec_i, ix_i = test_sample
x_init = (states_i[0,0,:] * data_stats['states_std'][0]) + (data_stats['states_mean'][0])
x_final = (goal_i[0,0,:] * data_stats['goal_std'][0]) + (data_stats['goal_mean'][0])'''
#x_init, x_final = sample_init_target()
test_sample[0][0,:,:] = (torch.tensor(np.repeat(x_init[None,:], 200, axis=0)) - data_stats['states_mean'])/(data_stats['states_std'] + 1e-6)#(torch.tensor(xs[:-1,:]) - data_stats['states_mean'])/(data_stats['states_std'] + 1e-6)#
test_sample[1][0,:,:] = torch.zeros((200,3))#(torch.tensor(us) - data_stats['actions_mean'])/(data_stats['actions_std'] + 1e-6)#
test_sample[2][0,:,0] = torch.zeros((200,))#torch.from_numpy(compute_reward_to_go(test_sample[1][0,:,:]))#
test_sample[3][:,0] = torch.zeros((200,))#torch.from_numpy(compute_constraint_to_go(test_sample[0][0,:,:].cpu().numpy(), obs_positions, obs_radii))#
test_sample[4][:,0] = torch.zeros((200,))
test_sample[5][0,:,:] = (torch.tensor(np.repeat(x_final[None,:], 200, axis=0)) - data_stats['goal_mean'])/(data_stats['goal_std'] + 1e-6)
print(test_sample[-1])
DT_trajectory = DT_manager.use_model_for_imitation_learning(model, train_loader, test_sample, ctg_perc=0., ttg=20.0, use_dynamics=True, ctg_clipped=True, chunksize=import_config['chunksize'])

[0]


/home/davide.celestini/ff_ws/src/freeflyer2/ff_control/transformer_controller/decision_transformer/../decision_transformer/manage_time.py:455: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ttgs_ol = torch.tensor((ttg - data_stats['ttgs_mean'][0])/(data_stats['ttgs_std'][0]+1e-6)).view(1, 1, 1).float().to(device)
/home/davide.celestini/ff_ws/src/freeflyer2/ff_control/transformer_controller/decision_transformer/../decision_transformer/manage_time.py:479: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ttgs_dyn = torch.tensor((ttg - data_stats['ttgs_mean'][0])/(data_stats['ttgs_std'][0]+1e-6)).view(1, 1, 1).float().to(device)
/home/davide.celestini/ff_ws/src/freeflyer2/ff_control/transformer_

In [8]:
states_ws_DT = np.hstack((DT_trajectory['xypsi_dyn'][:,:int(20.0/0.5)], x_final.reshape(-1,1))) # set warm start
actions_ws_DT = DT_trajectory['dv_dyn'][:,:int(20.0/0.5)]
np.sum(np.linalg.norm(actions_ws_DT, ord=1, axis=0))

0.10276204719701498

In [9]:
traj_scp_TTO, J_scp_TTO, status_scp_TTO, scp_TTO_iter = ocp_obstacle_avoidance(qm, states_ws_DT, actions_ws_DT, x_init, x_final)

ModuleNotFoundError: No module named 'dynamics.quadrotor'

In [ ]:
model2 = DT_manager.get_DT_model('checkpoint_quad_random_forest_ctgrtg', train_loader, eval_loader)
model2.eval();
DT_trajectory2,_ = DT_manager.use_model_for_imitation_learning(model2, train_loader, test_sample, ctg_perc=1., use_dynamics=True, output_attentions=True)

In [ ]:
from dynamics.quadrotor import *
qm = QuadModel(verbose=True)
xs, us, J, status = ocp_no_obstacle_avoidance(qm,x_init,x_final,initial_guess='line')
rtg_0 = -np.sum((np.linalg.norm(us.T, axis=0))**2)/2
dyn_trajectory, runtime_DT = DT_manager.torch_model_inference_dyn(model, test_loader, test_sample, rtg_perc=1., ctg_perc=0., rtg=rtg_0, ctg_clipped=True)

In [ ]:
import matplotlib.pyplot as plt
k = 1
plt.figure()
plt.plot(dyn_trajectory['xyz_dyn'][k,:])
plt.plot(DT_trajectory['xyz_dyn'][k,:])
plt.show()

In [ ]:
# Print the index of the trajectory and plot
DT_trajectory['xyz_ol'] = dyn_trajectory['xyz_dyn']
DT_trajectory['dv_ol'] = dyn_trajectory['dv_dyn']
print(test_sample[-1])
DT_manager.plot_DT_trajectory(DT_trajectory)

In [ ]:
dist, viol = check_koz_constraint(dyn_trajectory['xyz_dyn'].T,obs_positions,obs_radii)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(dist.T)
plt.plot([0, 100],[0, 0],'r-')
plt.xlabel('time')
plt.ylabel('distance from obstacles')
plt.show()

In [ ]:
states_ws_DT = np.append(dyn_trajectory['xyz_dyn'], (dyn_trajectory['xyz_dyn'][:,-1] + qm.f(dyn_trajectory['xyz_dyn'][:, -1], dyn_trajectory['dv_dyn'][:, -1])*dt).reshape((6,1)), 1).T
xs, us, J, status, scp_iter = ocp_obstacle_avoidance(qm, states_ws_DT, dyn_trajectory['dv_dyn'].T, x_init, x_final)

In [ ]:
xs2, us2, J, status = ocp_no_obstacle_avoidance(qm, x_init, x_final, initial_guess='keep')
print('-----------------')
xs2, us2, J, status, scp_iter = ocp_obstacle_avoidance(qm, xs2, us2, x_init, x_final)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10., azim=-20)

ax.scatter(x_init[0], x_init[1], x_init[2], color='k')
ax.scatter(x_final[0], x_final[1], x_final[2], color='k')
#ax.scatter(xs[8,0], xs[8,1], xs[8,2], color='k')

for i in range(n_obs):
    p = obs_positions[i]
    r = obs_radii[i]
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = p[0] + r * np.outer(np.cos(u), np.sin(v))
    y = p[1] + r * np.outer(np.sin(u), np.sin(v))
    z = p[2] + r * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, rstride=1, cstride=1, color='r', linewidth=0, alpha=0.3)
ax.plot3D(xs[:,0], xs[:,1], xs[:,2], color='b', linewidth=1)
ax.plot3D(xs2[:,0], xs2[:,1], xs2[:,2], color='g', linewidth=1)

plt.show()